In [5]:
# Preparar entorno
import sqlite3
import pandas as pd

In [6]:
# Conectar a la base de datos
conn = sqlite3.connect('database.db')
cursor = conn.cursor()

In [7]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):
    return pd.read_sql(query, conn)

In [8]:
#Creamos la tabla Reparto
cursor.execute ("""
CREATE TABLE IF NOT EXISTS Reparto (
    CodE INTEGER,
    CodB INTEGER,
    CodC INTEGER,
    Fecha DATE,
    Cantidad INTEGER,  
    PRIMARY KEY (CodE, CodB, CodC),
    FOREIGN KEY (CodE) REFERENCES Empleados(CodE),
    FOREIGN KEY (CodB) REFERENCES Bares(CodB),
    FOREIGN KEY (CodC) REFERENCES Cervezas(CodC)
)
""")

#Datos a insertar

valores=[
     (1, 1, 1, '2005-10-21', 240),
     (1, 1, 2, '2005-10-21', 48),
     (1, 2, 3, '2005-10-22', 60),
     (1, 4, 5, '2005-10-22', 4),
     (2, 2, 3, '2005-10-22', 48),
     (2, 2, 5, '2005-10-23', 42),
     (2, 4, 1, '2005-10-23', 480),
     (2, 4, 2, '2005-10-24', 72),
     (3, 3, 3, '2005-10-24', 48),
     (3, 3, 4, '2005-10-25', 20),
]
#Insertamos los datos
cursor.executemany("""
INSERT OR IGNORE INTO Reparto (CodE, CodB, CodC, Fecha, Cantidad)
VALUES (?, ?, ?, ?, ?)
""", valores)

#Confirmamos cambios
conn.commit()




In [9]:
query = '''
SELECT *
FROM Reparto
'''

df = sql_query(query)
df

,CodE,CodB,CodC,Fecha,Cantidad
0,1,1,1,2005-10-21,20
1,1,1,2,2005-10-21,48
2,1,2,3,2005-10-22,60
3,1,4,5,2005-10-22,4
4,2,2,3,2005-10-22,48
5,2,2,5,2005-10-23,42
6,2,4,1,2005-10-23,480
7,2,4,2,2005-10-24,72
8,3,3,3,2005-10-24,48
9,3,3,4,2005-10-25,20


In [10]:
#Creamos la tabla Empleados
cursor.execute ("""
CREATE TABLE IF NOT EXISTS Empleados (
    CodE INTEGER PRIMARY KEY,
    Nombre TEXT,
    Sueldo INTEGER
)
""")
#Datos a insertar

valores=[
     (1, "Carlos Lopez", 120000),
     (2, "Rosa Perez", 110000),
     (3, "Luisa Garcia", 100000),
  
]
#Insertamos los datos
cursor.executemany("""
INSERT OR IGNORE INTO Empleados (CodE, Nombre,Sueldo)
VALUES (?, ?, ?)
""", valores)

#Confirmamos cambios
conn.commit()


In [11]:
query = '''
SELECT *
FROM Empleados
'''

df = sql_query(query)
df

,CodE,Nombre,Sueldo
0,1,Carlos Lopez,120000
1,2,Rosa Perez,110000
2,3,Luisa Garcia,100000
3,1,Carlos Lopez,120000
4,2,Rosa Perez,110000
5,3,Luisa Garcia,100000
6,1,Carlos Lopez,120000
7,2,Rosa Perez,110000
8,3,Luisa Garcia,100000


In [12]:
#Creamos la tabla Bares
cursor.execute ("""
CREATE TABLE IF NOT EXISTS Bares (
    CodB INTEGER PRIMARY KEY,
    Nombre TEXT,
    Cif TEXT,
    Localidad TEXT
)
""")
#Datos a insertar

valores=[
     (1, "Stop", "11111111X", "Villa Botijo"),
     (2, "Las Vegas", "22222222Y", "Villa Botijo"),
     (3, "Club Social","", "Las Ranas"),
     (4, "Otra Ronda","33333333Z ", "La Esponja"),
  
]
#Insertamos los datos
cursor.executemany("""
INSERT OR IGNORE INTO Bares (CodB, Nombre,Cif,Localidad)
VALUES (?, ?, ?, ?)
""", valores)

#Confirmamos cambios
conn.commit()

In [13]:
query = '''
SELECT *
FROM Bares
'''

df = sql_query(query)
df

,CodB,Nombre,Cif,Localidad
0,1,Stop,11111111X,Villa Botijo
1,2,Las Vegas,22222222Y,Villa Botijo
2,3,Club Social,,Las Ranas
3,4,Otra Ronda,33333333Z,La Esponja


In [14]:
#Creamos la tabla Cervezas
cursor.execute ("""
CREATE TABLE IF NOT EXISTS Cervezas (
    CodC INTEGER PRIMARY KEY,
    Envase TEXT,
    Capacidad REAL,
    Stock INTEGER
)
""")

#Datos a insertar

valores=[
     (1, "botella", 0.2 , 3600),
     (2, "botella", 0.33 , 1200),
     (3, "lata", 0.33 , 2400),
     (4, "botella", 1 , 288),
     (5, "barril", 60 , 30),
    
]
#Insertamos los datos
cursor.executemany("""
INSERT OR IGNORE INTO Cervezas (CodC, Envase ,Capacidad , Stock)
VALUES (?, ?, ?, ?)
""", valores)

#Confirmamos cambios
conn.commit()

In [15]:
query = '''
SELECT *
FROM Cervezas
'''

df = sql_query(query)
df

,CodC,Envase,Capacidad,Stock
0,1,botella,0.20,3600
1,2,botella,0.33,1200
2,3,lata,0.33,2400
3,4,botella,1.00,288
4,5,barril,60.00,30


In [18]:
#1)Obtener  el  nombre    de  los  empleados   que  hayan  repartido  al  bar  Stop  durante la semana 
#del 17 al 23 de octubre de 2005
query = '''
SELECT DISTINCT e.Nombre, r.CodB, r.Fecha
FROM Reparto as r
JOIN Empleados as e ON r.CodE = e.CodE
WHERE r.CodB = '1'
AND r.Fecha BETWEEN '2005-10-17' AND '2005-10-23';
'''
df = sql_query(query)
df  

,Nombre,CodB,Fecha
0,Carlos Lopez,1,2005-10-21


In [19]:
#Obtener  el   Cif  y  nombre  de  los  bares  a  los  que  se  ha  repartido  cerveza  de  tipo  Botella  y  
#capacidad inferior a 1 litro, ordenados por localidad 
query = '''
SELECT b.Cif, b.Nombre
FROM Reparto AS r
JOIN Bares AS b ON r.CodB = b.CodB
JOIN Cervezas AS c ON r.CodC = c.CodC
WHERE Envase = "botella"
AND Capacidad < 1.00
ORDER BY b.Localidad;
'''
df = sql_query(query)
df 

,Cif,Nombre
0,33333333Z,Otra Ronda
1,33333333Z,Otra Ronda
2,11111111X,Stop
3,11111111X,Stop


In [20]:
#3)Obtener  los  repartos  (nombre  del  bar,  envase  y  capacidad  de  la  bebida, fecha y cantidad) realizados por Carlos Lopez

query = '''
SELECT b.Nombre, c.Envase, c.Capacidad, r.Fecha, r.Cantidad
FROM Reparto AS r
JOIN Empleados AS e ON r.CodE = e.CodE
JOIN Bares AS b ON r.CodB = b.CodB
JOIN Cervezas AS c ON r.CodC = c.CodC
WHERE e.Nombre LIKE "%Carlos Lopez%"
'''
df = sql_query(query)
df 


,Nombre,Envase,Capacidad,Fecha,Cantidad
0,Stop,botella,0.20,2005-10-21,20
1,Stop,botella,0.20,2005-10-21,20
2,Stop,botella,0.20,2005-10-21,20
3,Stop,botella,0.33,2005-10-21,48
4,Stop,botella,0.33,2005-10-21,48
5,Stop,botella,0.33,2005-10-21,48
6,Las Vegas,lata,0.33,2005-10-22,60
7,Las Vegas,lata,0.33,2005-10-22,60
8,Las Vegas,lata,0.33,2005-10-22,60
9,Otra Ronda,barril,60.00,2005-10-22,4


In [21]:
#4)Obtener  los  bares  a  los  que  se  les  ha  repartido  envases  de  tipo  botella  y  capacidad  0.2  ó  
#0.33
query = '''
SELECT b.Nombre
FROM Reparto AS r
JOIN Bares AS b ON r.CodB = b.CodB
JOIN Cervezas AS c ON r.CodC = c.CodC
WHERE c.Envase LIKE "%botella%"
AND c.Capacidad BETWEEN 0.2 AND 0.33
'''
df = sql_query(query)
df 

,Nombre
0,Stop
1,Stop
2,Otra Ronda
3,Otra Ronda


In [25]:
#5)Nombre de los empleados que han repartido a los bares "Stop" y "Las Vegas" cervezas con 
#envase botella.
query = '''
SELECT e.Nombre
FROM Reparto AS r
JOIN Empleados AS e ON r.CodE = e.CodE
JOIN Bares AS b ON r.CodB = b.CodB
JOIN Cervezas AS c ON r.CodC = c.CodC
WHERE (b.Nombre = "%Stop%") OR (b.Nombre= "%Las Vegas%")
AND c.Envase = "%botella%"
'''
df = sql_query(query)
df 

,Nombre


In [29]:
#6)Obtener  el  nombre  y  número  de  viajes  que  ha  realizado  cada  empleado  fuera  de  Villa  
#Botijo
query = '''
SELECT e.Nombre, COUNT(*) AS "Numero de Viajes"
FROM Reparto AS r
JOIN Empleados AS e ON r.CodE = e.CodE
JOIN Bares AS b ON r.CodB = b.CodB
WHERE b.Localidad != "Villa Botijo"
GROUP BY e.Nombre

'''
df = sql_query(query)
df 

,Nombre,Numero de Viajes
0,Carlos Lopez,3
1,Luisa Garcia,6
2,Rosa Perez,6


In [30]:
#7)Obtener el nombre y localidad del bar que más litros de cerveza ha comprado
query = '''
SELECT b.Nombre, b.Localidad
FROM Reparto AS r
JOIN Cervezas AS c ON r.CodC = c.CodC
JOIN Bares AS b ON r.CodB = b.CodB 
GROUP BY b.CodB, b.Nombre, b.Localidad --Agrupa datos por bar para representar cada bar
ORDER BY SUM(r.Cantidad * c.Capacidad) DESC --totaliza los litros por cada bar y devuelve solo el bar que mas litros ha comprado
LIMIT 1; -- DESC pon 1º el bar con más litros y limitamos a 1
'''
df = sql_query(query)
df 

,Nombre,Localidad
0,Las Vegas,Villa Botijo


In [34]:
#8)Obtener  los  bares  que  han  adquirido  todos  los  tipos  de  cerveza  con  envase  de  botella   y 
#capacidad menor que 1 litro
query = '''
SELECT b.Nombre
FROM Reparto AS r
JOIN Cervezas AS c ON r.CodC = c.CodC
JOIN Bares AS b ON r.CodB = b.CodB 
WHERE c.Envase = "botella"
AND c.Capacidad < 1.00

GROUP BY b.CodB, b.Nombre
HAVING COUNT(DISTINCT c.CodC) = (
    SELECT COUNT(*)
    FROM Cervezas
    WHERE Envase = 'botella' AND Capacidad < 1.0
    );
'''
df = sql_query(query)
df 

,Nombre
0,Stop
1,Otra Ronda


In [ ]:
#9)Subir un 5% el sueldo del empleado que más días haya trabajado. 
query = '''


'''
df = sql_query(query)
df 

TypeError: 'NoneType' object is not iterable